In [1]:
!pip install -q ultralytics

In [2]:
!pip install -q wandb

In [3]:
#!rm -rf /kaggle/working/*

In [4]:
import numpy as np 
import pandas as pd 
import os
from ultralytics import YOLO
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import cv2
import tensorflow 
import keras  
import glob 
from skimage import io 
import skimage
%matplotlib inline

In [5]:
from ultralytics.data.converter import convert_coco
convert_coco(labels_dir="/kaggle/input/cotton-seg/annotations", use_segments=True)

Annotations /kaggle/input/cotton-seg/annotations/instances_default.json: 100%|██████████| 83/83 [00:00<00:00, 17486.80it/s]

COCO data converted successfully.
Results saved to /kaggle/working/coco_converted


In [6]:
!cp /kaggle/input/cotton-seg/images/default/* /kaggle/working/coco_converted/images

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [7]:
from ultralytics.data.utils import autosplit
autosplit(path='/kaggle/working/coco_converted/images',weights=(0.7, 0.2, 0.1),annotated_only=False,)

Autosplitting images from /kaggle/working/coco_converted/images


100%|██████████| 2007/2007 [00:00<00:00, 18897.46it/s]


In [8]:
!mkdir /kaggle/working/coco_converted/images/train
!mkdir /kaggle/working/coco_converted/images/test
!mkdir /kaggle/working/coco_converted/images/val

In [9]:
!mkdir /kaggle/working/coco_converted/labels/train
!mkdir /kaggle/working/coco_converted/labels/test
!mkdir /kaggle/working/coco_converted/labels/val

In [13]:
!mv /kaggle/working/coco_converted/images/test/frame_033831.jpg /kaggle/working/coco_converted/images/frame_033831.jpg

In [14]:
root = '/kaggle/working/coco_converted/images'
dest = '/kaggle/working/coco_converted/images/test'
with open('/kaggle/working/coco_converted/autosplit_test.txt', 'r') as file:
    line = file.readline()
    while line:
        line.strip()
        path = root + line[8:-1]
        new_path = dest+line[8:-1]
        shutil.move(path, new_path)
        line = file.readline()
root = '/kaggle/working/coco_converted/images'
dest = '/kaggle/working/coco_converted/images/train'
with open('/kaggle/working/coco_converted/autosplit_train.txt', 'r') as file:
    line = file.readline()
    while line:
        line.strip()
        path = root + line[8:-1]
        new_path = dest+line[8:-1]
        shutil.move(path, new_path)
        line = file.readline()
        
root = '/kaggle/working/coco_converted/images'
dest = '/kaggle/working/coco_converted/images/val'
with open('/kaggle/working/coco_converted/autosplit_val.txt', 'r') as file:
    line = file.readline()
    while line:
        line.strip()
        path = root + line[8:-1]
        new_path = dest+line[8:-1]
        shutil.move(path, new_path)
        line = file.readline()

In [15]:
!mv /kaggle/working/coco_converted/labels/default/* /kaggle/working/coco_converted/labels

In [28]:
import glob as glob
root = '/kaggle/working/coco_converted/labels'
dest = '/kaggle/working/coco_converted/lables/test'
with open('/kaggle/working/coco_converted/autosplit_test.txt', 'r') as file:
    line = file.readline()
    while line:
        line.strip()
        line = line[8:-4]
        path = root + line
        new_path = dest
        for file in glob.glob(line+'*'):
            shutil.move(file, new_path)
        line = file.readline()

In [ ]:
import wandb
wandb.login()
model = YOLO("yolov8n-seg.pt")
#results = model.train(data="/kaggle/input/laboro-yaml-6/laboro-tomato.yaml", epochs=100, imgsz=640, device=[0,1])
results = model.train(data="/kaggle/input/cotton-seg-yaml/cotton-seg.yaml", epochs=10, imgsz=640, device=[0,1])

In [ ]:
#validation_results = model.val(data="/kaggle/input/laboro-yaml-6/laboro-tomato.yaml", imgsz=640, batch=16, conf=0.25, iou=0.6, device="0")
validation_results = model.val(data="/kaggle/input/laboro-tomato-gray/laboro-tomato-gray.yaml", imgsz=640, batch=16, conf=0.25, iou=0.6, device="0")

In [ ]:
import cv2
import urllib.request
url = 'https://bonnieplants.com/cdn/shop/articles/BONNIE_tomatoes_iStock-481349128-1800px_9f8f5390-a418-4d91-a3d0-00ae0b7900cb.jpg'
urllib.request.urlretrieve(url, 'test.jpg')
results = model.predict('test.jpg')


In [ ]:
print(results[0].masks)
#im_bgr = result[0].masks()  # BGR-order numpy array
#im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
#plt.imshow(im_rgb)
#plt.show()